# IMPORTING REQUIRED LIBRARIES

In [1]:
from selenium import webdriver                          #For opening browser       
import pandas as pd                                     #For creating dataframe
from bs4 import BeautifulSoup                           #For scraping data
import time                                             #For waiting
import numpy as np                                      #For array
from tqdm import tqdm                                   #For progress bar

# OPENING REQUIRED SITE IN BROWSER

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://m.imdb.com/chart/starmeter/')

# GETTING HTML CODE OF OPENED WEBPAGE

In [4]:
html = driver.page_source

# CONVERTING EXTRACTED HTML CODE INTO SOUP TO SCRAP DATA

In [5]:
soup = BeautifulSoup(html,"html.parser")

# EXTRACTING NAMES OF CELEBRITIES AND LINKS TO THEIR INDIVIDUAL WEBPAGES

In [6]:
a_list = soup.find_all("a")
name_list = []
link_list = []
for i in range(43,343,3):
    name_list.append(a_list[i].get('aria-label'))
    link_list.append(a_list[i].get('href'))

# EXTRACTING THE FAMOUS WORK OF THESE CELEBRITIES

In [7]:
famous_list = []
a_list1 = soup.findAll('a',{'data-testid':'nlib-known-for-title'})
for i in a_list1:
    famous_list.append(i.text)

# FUNCTION TO EXTRACT DATE OF BIRTH OF THE CELEBRITIES

In [8]:
def get_dob(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        dob = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[0].text
    except Exception as e:
        dob = np.nan
    return dob

# FUNCTION TO EXTRACT HEIGHT OF THE CELEBRITIES

In [9]:
def get_height(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        height = sec.findAll('li',{'data-testid':'nm_pd_he'})[0].findAll('li')[0].text
    except Exception as e:
        height = np.nan
    return height

# FUNCTION TO EXTRACT BIRTHPLACE OF THE CELEBRITIES

In [10]:
def get_birthplace(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        bp = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[1].text
    except Exception as e:
        bp = np.nan
    return bp

# FUNCTION TO EXTRACT AWARD DATA OF THE CELEBRITIES

In [11]:
def get_awards(soup):
    try:
        div_list = soup.findAll('div', {'data-testid':'awards'})
        awards = div_list[0].findAll('li',{'class':'ipc-inline-list__item'})[0].text
    except Exception as e:
        awards = np.nan
    return awards

# FUNCTION TO EXTRACT ROLES OF THE CELEBRITIES

In [12]:
def get_role(soup):
    role_list = []
    try:
        li_list = soup.findAll('ul', {'class':'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'})[0].findAll('li')
        for i in li_list:
            role_list.append(i.text)
    except Exception as e:
        role_list.append(np.nan)
    return role_list

In [15]:
get_role(soup)

['Actor', 'Director', 'Writer']

# MAIN LOOP TO ITERATE THROUGH EACH CELEBRITY'S WEBPAGE AND EXTRACT DATA

In [16]:
dob_list = []
height_list = []
role_list = []
award_list = []
birth_list = []
for i in tqdm(link_list, desc="Processing Links"):
    driver.get(f'https://m.imdb.com/{i}')
    #time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    dob_list.append(get_dob(soup))
    height_list.append(get_height(soup))
    birth_list.append(get_birthplace(soup))
    role_list.append(get_role(soup))
    award_list.append(get_awards(soup))

Processing Links:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Links: 100%|██████████| 100/100 [10:52<00:00,  6.52s/it]


# CREATING DATAFRAME FOR COLLECTED DATASET 

In [17]:
df = pd.DataFrame({
    'Rank':np.arange(1,101),
    'Name': name_list,
    'Date_of_birth': dob_list,
    'height': height_list,
    'Role': role_list,
    'Awards': award_list,
    'Famous_for': famous_list,
    'Birth_place': birth_list
})

# CHECKING THE DATASET

In [18]:
df.head()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Sofia Boutella,"April 3, 1982",5′ 5″ (1.65 m),"[Actress, Soundtrack]",1 win & 2 nominations,Kingsman: The Secret Service (2014),"Bab El Oued, Algeria"
1,2,Ed Skrein,"March 29, 1983",6′ 1″ (1.85 m),"[Actor, Director, Writer]",2 wins & 2 nominations,Deadpool (2016),"Camden, London, England, UK"
2,3,Barry Keoghan,"October 18, 1992",5′ 8″ (1.73 m),[Actor],11 wins & 76 nominations total,The Killing of a Sacred Deer (2017),"Dublin, Ireland"
3,4,Serinda Swan,"July 11, 1984",5′ 7″ (1.70 m),"[Actress, Producer, Director]",2 nominations,Coroner (2019–2022),"West Vancouver, British Columbia, Canada"
4,5,Alan Ritchson,"November 28, 1982",6′ 3″ (1.91 m),"[Actor, Producer, Director]",3 wins & 3 nominations,Teenage Mutant Ninja Turtles (2014),"Grand Forks, North Dakota, USA"


In [19]:
df.tail()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
95,96,Kristin Minter,"November 22, 1965",5′ 8″ (1.73 m),[Actress],1 nomination,ER (1995–2003),"Miami, Florida, USA"
96,97,Zac Efron,"October 18, 1987",5′ 8″ (1.73 m),"[Actor, Producer, Soundtrack]",35 wins & 44 nominations,17 Again (2009),"San Luis Obispo, California, USA"
97,98,Emma Corrin,"December 13, 1995",5′ 8″ (1.73 m),"[Actress, Music Department, Soundtrack]",10 wins & 10 nominations total,Lady Chatterley's Lover (2022),"Royal Tunbridge Wells, Kent, England, UK"
98,99,Kelly Reilly,"July 18, 1977",5′ 6″ (1.68 m),"[Actress, Producer]",7 wins & 12 nominations,Pride & Prejudice (2005),"Surrey, England, UK"
99,100,Kamar de los Reyes,"November 8, 1967",5′ 10″ (1.78 m),"[Actor, Writer, Producer]",1 win & 6 nominations,Sleepy Hollow (2017– ),"San Juan, Puerto Rico"


In [20]:
df.sample(20)

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
86,87,Jennifer Connelly,"December 12, 1970",5′ 6½″ (1.69 m),"[Actress, Producer]",16 wins & 29 nominations total,A Beautiful Mind (2001),"Cairo, New York, USA"
75,76,Alan Rickman,"February 21, 1946",6′ 1″ (1.85 m),"[Actor, Writer, Director]",19 wins & 38 nominations total,Die Hard (1988),"Hammersmith, London, England, UK"
34,35,Glen Powell,"October 21, 1988",6′ (1.83 m),"[Actor, Producer, Writer]",3 wins & 4 nominations,Hidden Figures (2016),"Austin, Texas, USA"
80,81,Anita Dobson,"April 29, 1949",5′ 3″ (1.60 m),[Actress],NaN,Red Dwarf (1993– ),"Stepney, London, England, UK"
0,1,Sofia Boutella,"April 3, 1982",5′ 5″ (1.65 m),"[Actress, Soundtrack]",1 win & 2 nominations,Kingsman: The Secret Service (2014),"Bab El Oued, Algeria"
49,50,Myha'la,"April 6, 1996",5′ 1″ (1.55 m),[Actress],NaN,Industry (2020–2022),"California, USA"
3,4,Serinda Swan,"July 11, 1984",5′ 7″ (1.70 m),"[Actress, Producer, Director]",2 nominations,Coroner (2019–2022),"West Vancouver, British Columbia, Canada"
81,82,Rebecca Ferguson,"October 19, 1983",5′ 5″ (1.65 m),"[Actress, Producer, Soundtrack]",9 wins & 20 nominations,The Greatest Showman (2017),Sweden
38,39,Emma Stone,"November 6, 1988",5′ 6″ (1.68 m),"[Actress, Producer, Soundtrack]",82 wins & 214 nominations total,La La Land (2016),"Scottsdale, Arizona, USA"
1,2,Ed Skrein,"March 29, 1983",6′ 1″ (1.85 m),"[Actor, Director, Writer]",2 wins & 2 nominations,Deadpool (2016),"Camden, London, England, UK"


# SAVING THE DATASET

In [21]:
df.to_csv('celebrity_data.csv',index= False)